# This notebook must be run in the GitHub repository published in Bee et al. 2021, along with the `.h5` files included in this directory

In [2]:
%pylab notebook

# # Run on CPU not GPU
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
from tensorflow.keras import layers

import cupyck

import primo.models
import primo.datasets
# import primo.tools.filepath as filepaths
import primo.tools.sequences as seqtools
import pandas as pd


Populating the interactive namespace from numpy and matplotlib


In [3]:
sess_args = {
    "max_seqlen": 240
}
try:
    session = cupyck.GPUSession(**sess_args)
except RuntimeError:
    print "GPU startup failed. falling back to multicore backend."
    session = cupyck.MulticoreSession()
simulator = primo.models.Simulator(session)

In [4]:
# Load the model to be examined
encoder_20 = tf.keras.models.load_model('/tf/primo/data/models/encoder_model_20_3.h5')
encoder_80 = tf.keras.models.load_model('/tf/primo/data/models/encoder_model_80_3.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
### 
# Define utilities
###

def confidence_by_base(sequences):
    """Takes a batch of softmax sequences, and returns vector of average confidence in each position
    
    sequences: Number of exmaple x Number of bases x 4
    
    Returns: 1 x number of bases
    """
    # reduce last dimension to max base number
    return np.mean(np.max(sequences, axis=-1), axis=0)

def entropy_by_base(sequences):
    x = np.sum(sequences, axis=0)
    x /= np.sum(x, axis=1)[:, None]
    entropy = -np.sum(x * np.log(x + 1e-10), axis=1)
    return entropy

def base_distribution(sequences):
    one_hots = one_hot(sequences)
    bases = np.sum(one_hots, axis=0)

def one_hot(nparray, depth = 0, on_value = 1, off_value = 0):
    if depth == 0:
        depth = np.max(nparray) + 1
    assert np.max(nparray) < depth, "the max index of nparray: {} is larger than depth: {}".format(np.max(nparray), depth)
    shape = nparray.shape
    out = np.ones(shape + (depth,), dtype=np.float32) * off_value
    indices = []
    for i in range(nparray.ndim):
        tiles = [1] * nparray.ndim
        s = [1] * nparray.ndim
        s[i] = -1
        r = np.arange(shape[i]).reshape(s)
        if i > 0:
            tiles[i-1] = shape[i-1]
            r = np.tile(r, tiles)
        indices.append(r)
    indices.append(nparray)
    out[tuple(indices)] = on_value
    return out

def get_multiple_batchs(n, generator):
    indices = None
    values = None
    for _ in range(n):
        i, v = next(generator)
        if indices is None:
            indices = i
            values = v
        else:
            indices = np.concatenate([indices, i])
            values = np.concatenate([values, v])
    return indices, values

In [6]:
# ###
# # Load queries and see what they encode to
# ###
# query_features = pd.read_hdf('/tf/primo/data/queries/features.h5')
# query_seqs = seqtools.onehots_to_seqs(encoder.predict(query_features))
# print("Query Seqs: \n" + "\n".join(query_seqs))

# def seq_str_to_input(seq):
#     return np.transpose(seqtools.seqs_to_onehots(seq), [1, 0, 2])

In [7]:
###
# Use data batch generators to get a batch of random training and validation
# data
###encoder_1site_triplet_20220128_1

def keras_batch_generator(dataset_batch_generator, similarity_threshold):
    # Yield datasets
    while True:
        # This tuple contains:
        # indices: a positive integer uniquely identifying an image. This index is obtained by enumerating all the images in the dataset (before splitting them into test/train/validate datasets)
        # pairs:
        indices, pairs = next(dataset_batch_generator)
        # The Euclidean distances between the two vectors in each pair
        distances = np.sqrt(np.square(pairs[:,0,:] - pairs[:,1,:]).sum(1))
        # Whether or not the images in this pair should be considered 'similar'. This is a boolean value, represented by an int (0 or 1), and is determined by whether the aforementioned Euclidean distances between image feature vectors are under some pre-deterined "similarity threshold".
        similar = (distances < similarity_threshold).astype(int)
        # Yield a pair of sequences, and 0-or-1 indicating whether they're similar.
        yield pairs, similar
        
batch_size = 200
similarity_threshold = 75

validation_dataset = primo.datasets.OpenImagesVal('/tf/open_images/validation/')

# Generators for balanced pairs
val_pair_generator = keras_batch_generator(
    validation_dataset.balanced_pairs(batch_size, similarity_threshold), 
    similarity_threshold)

# Get pairs (250 batches x 200 batch size = 50k pairs)
val_feat_pairs, val_truth = get_multiple_batchs(250, val_pair_generator) 


### 80 nt encoded

In [8]:
# Encode sequence pairs with the 80 nt hybridization encoder
val_seq_pairs = one_hot(np.stack([
    encoder_80.predict(val_feat_pairs[:, 0, :]).argmax(-1),
    encoder_80.predict(val_feat_pairs[:, 1, :]).argmax(-1),
], axis=1)).astype(np.float32)
    
# compare false positive returns to true returns
pairs_p = val_seq_pairs[val_truth > 0.5] #onehot vectors similar
pairs_n = val_seq_pairs[val_truth < 0.5] #onehot vectors dissimilar

# similar queries and targets
query_features_p = seqtools.onehots_to_seqs(pairs_p[:, 0, :]) 
target_features_p = seqtools.onehots_to_seqs(pairs_p[:, 1, :])

# dissimilar queries and targets
query_features_n = seqtools.onehots_to_seqs(pairs_n[:, 0, :])
target_features_n = seqtools.onehots_to_seqs(pairs_n[:, 1, :])

Yp = simulator.simulate(pd.DataFrame({
    'query_features': query_features_p,
    'target_features': target_features_p,
}))
Yn = simulator.simulate(pd.DataFrame({
    'query_features': query_features_n,
    'target_features': target_features_n,
}))

Yp_yield = Yp['duplex_yield']
Yn_yield = Yn['duplex_yield']

threshold_data = [] 
recall_data = []
false_positive_data = []

thresholds=1-np.logspace(-14, 0, 100)
for t in thresholds:
    recall = (Yp_yield > t).mean()
    false_positive = (Yn_yield > t).mean()
    threshold_data.append(t)
    recall_data.append(recall)
    false_positive_data.append(false_positive)
    
plt.figure()   
plt.scatter(false_positive_data, recall_data, c=threshold_data, cmap=plt.cm.viridis)
plt.ylabel("Proportion Correctly Retrieved")
plt.xlabel("Proportion Falsely Retrieved")
plt.colorbar(label="Yield Threshold")
plt.title("80nt Hybridization Model")
plt.tight_layout()
# plt.savefig('80_hyb_retrieval_random_pair_simulations.png', bbox_inches='tight', dpi=1000)
plt.show()

<IPython.core.display.Javascript object>

In [9]:
print list(thresholds)

[0.99999999999999, 0.9999999999999861, 0.9999999999999808, 0.9999999999999735, 0.9999999999999633, 0.999999999999949, 0.9999999999999295, 0.9999999999999023, 0.9999999999998647, 0.9999999999998126, 0.9999999999997405, 0.9999999999996406, 0.9999999999995023, 0.9999999999993108, 0.9999999999990454, 0.9999999999986781, 0.9999999999981692, 0.9999999999974646, 0.9999999999964888, 0.9999999999951374, 0.9999999999932658, 0.999999999990674, 0.9999999999870846, 0.9999999999821135, 0.9999999999752293, 0.9999999999656953, 0.9999999999524919, 0.9999999999342066, 0.9999999999088838, 0.9999999998738143, 0.9999999998252471, 0.9999999997579871, 0.9999999996648398, 0.9999999995358411, 0.9999999993571926, 0.9999999991097849, 0.9999999987671533, 0.9999999982926474, 0.9999999976355106, 0.9999999967254508, 0.9999999954651215, 0.9999999937197086, 0.9999999913025099, 0.9999999879549646, 0.9999999833189946, 0.999999976898703, 0.9999999680073286, 0.9999999556937854, 0.9999999386409273, 0.9999999150246565, 0.99

In [10]:
print false_positive_data

[0.004521085060414499, 0.004521085060414499, 0.004521085060414499, 0.004521085060414499, 0.004521085060414499, 0.004521085060414499, 0.004521085060414499, 0.004521085060414499, 0.004521085060414499, 0.004641113867328159, 0.004641113867328159, 0.004761142674241818, 0.0048811714811554776, 0.005001200288069136, 0.0050812194926782425, 0.0053212771065055614, 0.005401296311114668, 0.005721373129551092, 0.005881411538769305, 0.006121469152596623, 0.006361526766423942, 0.00660158438025126, 0.0067616227894694726, 0.006921661198687685, 0.007041690005601345, 0.00724173801712411, 0.007321757221733216, 0.007561814835560535, 0.00788189165399696, 0.008081939665519724, 0.00840201648395615, 0.00868208370008802, 0.009122189325438105, 0.009642314155397295, 0.00996239097383372, 0.010362486996879252, 0.011042650236056653, 0.011402736656797632, 0.011922861486756822, 0.012402976714411458, 0.012723053532847884, 0.013283187965111626, 0.014043370408898135, 0.014723533648075538, 0.015483716091862047, 0.016123869

In [11]:
print recall_data

[0.3416380068783492, 0.3416380068783492, 0.341797968487563, 0.341797968487563, 0.3422378629129009, 0.34227785331520433, 0.34283771894745263, 0.3432376229704871, 0.34371750779812843, 0.3451171718787491, 0.3465168359593698, 0.34787650963768696, 0.349756058545949, 0.35259537710949374, 0.3554346956730385, 0.3583539950411901, 0.36263296808765894, 0.36739182596176917, 0.3725505878589139, 0.3791889946412861, 0.3856674398144445, 0.3927457410221547, 0.3988642725745821, 0.4058226025753819, 0.4129408941853955, 0.4206190514276574, 0.4279372950491882, 0.43533551947532595, 0.44197392625769816, 0.4502519395345117, 0.4582900103975046, 0.465928177237463, 0.4732864112612973, 0.4812045109173798, 0.48848276413660724, 0.4972806526433656, 0.5053587139086619, 0.5126769575301927, 0.5195553067263856, 0.5277133487962888, 0.5343917459809646, 0.5432296248900264, 0.5508278013276814, 0.5579460929376949, 0.5656242501799568, 0.5738622730544669, 0.5813404782852115, 0.588898664320563, 0.5965768215628249, 0.604814844437

#### 20 nt encoder 

In [12]:
# Encode sequence pairs with the 20 nt hybridization encoder
val_seq_pairs = one_hot(np.stack([
    encoder_20.predict(val_feat_pairs[:, 0, :]).argmax(-1),
    encoder_20.predict(val_feat_pairs[:, 1, :]).argmax(-1),
], axis=1)).astype(np.float32)
    
# compare false positive returns to true returns
pairs_p = val_seq_pairs[val_truth > 0.5] #onehot vectors similar
pairs_n = val_seq_pairs[val_truth < 0.5] #onehot vectors dissimilar

# similar queries and targets
query_features_p = seqtools.onehots_to_seqs(pairs_p[:, 0, :]) 
target_features_p = seqtools.onehots_to_seqs(pairs_p[:, 1, :])

# dissimilar queries and targets
query_features_n = seqtools.onehots_to_seqs(pairs_n[:, 0, :])
target_features_n = seqtools.onehots_to_seqs(pairs_n[:, 1, :])

Yp = simulator.simulate(pd.DataFrame({
    'query_features': query_features_p,
    'target_features': target_features_p,
}))
Yn = simulator.simulate(pd.DataFrame({
    'query_features': query_features_n,
    'target_features': target_features_n,
}))

Yp_yield = Yp['duplex_yield']
Yn_yield = Yn['duplex_yield']

threshold_data = [] 
recall_data = []
false_positive_data = []

thresholds=1-np.logspace(-14, 0, 100)
for t in thresholds:
    recall = (Yp_yield > t).mean()
    false_positive = (Yn_yield > t).mean()
    threshold_data.append(t)
    recall_data.append(recall)
    false_positive_data.append(false_positive)
    
plt.figure()   
plt.scatter(false_positive_data, recall_data, c=threshold_data, cmap=plt.cm.viridis)
plt.ylabel("Proportion Correctly Retrieved")
plt.xlabel("Proportion Falsely Retrieved")
plt.colorbar(label="Yield Threshold")
plt.title("20nt Hybridization Model")
plt.tight_layout()
# plt.savefig('20_hyb_retrieval_random_pair_simulations.png', bbox_inches='tight', dpi=1000)
plt.show()

<IPython.core.display.Javascript object>

In [13]:
print list(thresholds)

[0.99999999999999, 0.9999999999999861, 0.9999999999999808, 0.9999999999999735, 0.9999999999999633, 0.999999999999949, 0.9999999999999295, 0.9999999999999023, 0.9999999999998647, 0.9999999999998126, 0.9999999999997405, 0.9999999999996406, 0.9999999999995023, 0.9999999999993108, 0.9999999999990454, 0.9999999999986781, 0.9999999999981692, 0.9999999999974646, 0.9999999999964888, 0.9999999999951374, 0.9999999999932658, 0.999999999990674, 0.9999999999870846, 0.9999999999821135, 0.9999999999752293, 0.9999999999656953, 0.9999999999524919, 0.9999999999342066, 0.9999999999088838, 0.9999999998738143, 0.9999999998252471, 0.9999999997579871, 0.9999999996648398, 0.9999999995358411, 0.9999999993571926, 0.9999999991097849, 0.9999999987671533, 0.9999999982926474, 0.9999999976355106, 0.9999999967254508, 0.9999999954651215, 0.9999999937197086, 0.9999999913025099, 0.9999999879549646, 0.9999999833189946, 0.999999976898703, 0.9999999680073286, 0.9999999556937854, 0.9999999386409273, 0.9999999150246565, 0.99

In [15]:
print false_positive_data

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.000960230455309e-05, 4.000960230455309e-05, 4.000960230455309e-05, 4.000960230455309e-05, 4.000960230455309e-05, 4.000960230455309e-05, 4.000960230455309e-05, 4.000960230455309e-05, 4.000960230455309e-05, 4.000960230455309e-05, 4.000960230455309e-05, 0.00012002880691365927, 0.00012002880691365927, 0.00012002880691365927, 0.00012002880691365927, 0.00012002880691365927, 0.00016003840921821236, 0.00016003840921821236, 0.00020004801152276547, 0.00024005761382731855, 0.00040009602304553094, 0.00044010562535008404, 0.0005601344322637432, 0.0007601824437865087, 0.0008001920460910619, 0.001240297671441146, 0.0015203648875730174, 0.0019604705129231017, 0.002280547331359526, 0.002920700968232376, 0.003800912218932544, 0.0044010562535008405, 0.005001200288069136, 0.005961430743378411, 0.006801632391774026, 0.007481795630951428

In [16]:
print recall_data

[0.00011997120691034152, 0.00011997120691034152, 0.00011997120691034152, 0.00011997120691034152, 0.00011997120691034152, 0.00011997120691034152, 0.00011997120691034152, 0.00011997120691034152, 0.00011997120691034152, 0.00011997120691034152, 0.00011997120691034152, 0.00011997120691034152, 0.00011997120691034152, 0.00011997120691034152, 0.00011997120691034152, 0.00011997120691034152, 0.00011997120691034152, 0.00015996160921378868, 0.00015996160921378868, 0.00027993281612413023, 0.00027993281612413023, 0.0004398944253379189, 0.0004398944253379189, 0.0007998080460689434, 0.000919779252979285, 0.0010797408621930737, 0.0015996160921378868, 0.0016795968967447813, 0.001879548908262017, 0.002479404942813725, 0.00339918419579301, 0.004278973046468847, 0.005558665920179157, 0.006838358793889466, 0.007878109253779093, 0.008757898104454931, 0.009557706150523874, 0.011277293449572103, 0.012836919139406542, 0.015156362473006479, 0.01867551787570983, 0.02151483643925458, 0.02519395345117172, 0.0285931

# OLD
This code was used to examine preliminary relationships, it has been refactored above

In [ ]:
###
# Use data batch generators to get a batch of random training and validation
# data
###encoder_1site_triplet_20220128_1

def keras_batch_generator(dataset_batch_generator, similarity_threshold):
    # Yield datasets
    # TODO: Verify with Callie this understanding is correct https://github.com/uwmisl/cas9-similarity-search/issues/2
    while True:
        # This tuple contains:
        # indices: a positive integer uniquely identifying an image. This index is obtained by enumerating all the images in the dataset (before splitting them into test/train/validate datasets)
        # pairs:
        indices, pairs = next(dataset_batch_generator)
        # The Euclidean distances between the two vectors in each pair
        distances = np.sqrt(np.square(pairs[:,0,:] - pairs[:,1,:]).sum(1))
        # Whether or not the images in this pair should be considered 'similar'. This is a boolean value, represented by an int (0 or 1), and is determined by whether the aforementioned Euclidean distances between image feature vectors are under some pre-deterined "similarity threshold".
        similar = (distances < similarity_threshold).astype(int)
        # Yield a pair of sequences, and 0-or-1 indicating whether they're similar.
        yield pairs, similar

batch_size = 200
similarity_threshold = 75
train_dataset = primo.datasets.OpenImagesTrain(
    '/tf/open_images/train/', switch_every=10**5
)
validation_dataset = primo.datasets.OpenImagesVal('/tf/open_images/validation/')
 
# # Generator for triplets
# train_triplet_generator = primo.datasets.dataset.triplet_batch_generator(
#     train_dataset.random_features(batch_size),
#     similarity_threshold
# )

# val_triplet_generator = primo.datasets.dataset.triplet_batch_generator(
#     validation_dataset.random_features(batch_size),
#     similarity_threshold
# )

# Generators for balanced pairs
train_pair_generator = keras_batch_generator(
    train_dataset.balanced_pairs(batch_size, similarity_threshold),
    similarity_threshold
)
val_pair_generator = keras_batch_generator(
    validation_dataset.balanced_pairs(batch_size, similarity_threshold),
    similarity_threshold
)

# batch = next(batch_generator)
# del batch_generator

# val_batch = next(encoder_val_batches)
# del encoder_val_batches

In [ ]:
def plot_pair_scatter(seq_pairs, y_true, title):
    pairs_p = seq_pairs[y_true > 0.5] #one hot vectors
    pairs_n = seq_pairs[y_true < 0.5]
    
#     print pairs_p.shape()
    # Compute the log10 scores for positive (similar) pairs
#     Yp = log_multisite_predictor(pairs_p).numpy()
    # Compute the log10 score for negative (dis-similar) pairs
#     Yn = log_multisite_predictor(pairs_n).numpy()
    
    query_features_p = seqtools.onehots_to_seqs(pairs_p[:, 0, :]) # reference
    target_features_p = seqtools.onehots_to_seqs(pairs_p[:, 1, :])
    query_features_n = seqtools.onehots_to_seqs(pairs_n[:, 0, :])
    target_features_n = seqtools.onehots_to_seqs(pairs_n[:, 1, :])
    Yp = simulator.simulate(pd.DataFrame({
        'query_features': query_features_p,
        'target_features': target_features_p,
    }))['duplex_yield']
    Yn = simulator.simulate(pd.DataFrame({
        'query_features': query_features_n,
        'target_features': target_features_n,
    }))['duplex_yield']
    
    recall = (Yp > 10**(-10.0)).mean()
    false_positive = (Yn > 10**(-10.0)).mean()
    
#     Yn=pairs_p[:, 1, :] # target
    
#     Yp = seqtools.onehots_to_seqs(Yp)
#     print Yp
#     Yn = seqtools.onehots_to_seqs(Yn)
#     #make ref and target into appropriate df
#     target_query_df = pd.DataFrame(list(zip(Yp, Yn)), columns = ['query_features', 'target_features'])
#     results = simulator.simulate(target_query_df) # creates df with last column 'duplex_yield'
#     duplex_yield = results["duplex_yield"]
#     target_query_df = target_query_df.join(duplex_yield)

    #     thresholds=1-np.logspace(-14, 0, 100)
#     recall = (Yp > 10**(-10.0)).mean()
#     false_positive = (Yn > 10**(-10.0)).mean()

#     print("recall: {}, false_positive: {}".format([recall, false_positive]))

    plt.figure()
    plt.plot(Yp, '.', label='pos')
    plt.plot(Yn, 'x', label='neg')
    plt.legend()
    plt.title('{} - recall: {:.2f}, false positive: {:.2f}'.format(title, recall, false_positive))

val_feat_pairs, val_truth = get_multiple_batchs(10, val_pair_generator)
val_seq_pairs = one_hot(np.stack([
    encoder.predict(val_feat_pairs[:, 0, :]).argmax(-1),
    encoder.predict(val_feat_pairs[:, 1, :]).argmax(-1),
], axis=1)).astype(np.float32)

train_feat_pairs, train_truth = get_multiple_batchs(10, train_pair_generator)
train_seq_pairs = one_hot(np.stack([
    encoder.predict(train_feat_pairs[:, 0, :]),
    encoder.predict(train_feat_pairs[:, 1, :]),
], axis=1).argmax(-1))

plot_pair_scatter(val_seq_pairs, val_truth, 'Validation Data')
plot_pair_scatter(train_seq_pairs, train_truth, 'Train Data')

In [ ]:
def nn_recall(scores, distances, k=10, n_thresholds=500, thresholds=None):
    
    if thresholds is None:
        thresholds = np.linspace(scores.min(), scores.max(), n_thresholds)
        thresholds[0] = (thresholds[0] + thresholds[1])/2
        
    retrieved = scores >= thresholds.reshape(-1,1)
    p_retrieved = retrieved.mean(1)
    
    neighbors = distances.argsort()[:k]
    
    nn_recall = retrieved[:, neighbors].mean(1)
    
    return nn_recall, p_retrieved, thresholds

n,p,t = nn_recall(
    df.duplex_yield.values/df.duplex_yield.max(),
    df.euclidean_distance.values,
    k=100,
    thresholds=1-np.logspace(-14, 0, 100)
)

plt.scatter(n,p,c=t, cmap=plt.cm.viridis_r)
plot_baselines(100, len(df))(plt.gca())
plt.xlabel("100-NN Recall")
plt.ylabel("Proportion Retrieved")
plt.colorbar(label="Yield Threshold")

In [ ]:
### Sequence Confidence and Variation
#
# Get a random set of validation features and encode to sequences, then 
# plot the confidence and variation (entropy) of the sequence pool
# for each sequence position. 

# _, example_features = next(validation_dataset.random_features(500))
# _, example_features = next(train_dataset.random_features(500))

seqs_softmax = encoder.predict(example_features)
seqs_onehot = one_hot(seqs_softmax.argmax(-1))

unique_seqs = np.unique(seqs_onehot, axis=0)
sequence_uniqueness = len(unique_seqs) / len(seqs_onehot)

base_distributions = seqs_onehot.sum(0) / len(seqs_onehot)
seqlen = seqs_onehot.shape[1]

confidence = confidence_by_base(seqs_softmax)
entropy = entropy_by_base(seqs_softmax)

seqlen = seqs_softmax.shape[1]
fig = plt.figure(figsize=[7, 7])
# fig.suptitle(f'Sequence Uniqueness = {sequence_uniqueness:0.2f}')
axes = fig.subplots(3,1)

ax = axes[0]
ax.bar(np.arange(0, seqlen), confidence)
ax.set_title('confidence')

ax = axes[1]
ax.bar(np.arange(0, seqlen), entropy)
ax.set_title('entropy')

ax = axes[2]
ax.set_title('base distribution')
ax.set_xlabel('sequence position')
bottom = np.zeros_like(base_distributions[:, 0])
ax.bar(np.arange(seqlen), base_distributions[:, 0], label='A', )
bottom += base_distributions[:, 0]
ax.bar(np.arange(seqlen), base_distributions[:, 1], label='T', bottom=bottom)
bottom += base_distributions[:, 1]
ax.bar(np.arange(seqlen), base_distributions[:, 2], label='C', bottom=bottom)
bottom += base_distributions[:, 2]
ax.bar(np.arange(seqlen), base_distributions[:, 3], label='G', bottom=bottom)
ax.legend(bbox_to_anchor=(1.15, 0.9))

plt.tight_layout()